<a href="https://colab.research.google.com/github/avhadrohini163/RICE_Final_Project/blob/main/Final_PySpark_code_creating_dataframe_from_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.4 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:13 http://archive.ubuntu.com/ubun

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-19 01:14:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.01MB/s    in 0.2s    

2022-03-19 01:14:42 (6.01 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:

# Start Spark session
from pyspark.sql import SparkSession, types
spark = SparkSession.builder.appName("RICE_Project").getOrCreate()




In [6]:
# Start a SparkSession
import findspark
import requests
import json
import sys
from pyspark.sql.functions import *
from pyspark.sql import functions as sf
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
import datetime as dt
from pyspark.sql import SparkSession
from pyspark import *
from pyspark.context import SparkContext
import pandas as pd
import pyspark
from urllib.request import Request,urlopen





In [7]:
spark = SparkSession.builder.appName("project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [8]:
http=urlopen('https://api.covid19api.com/summary').read().decode('utf-8')

In [9]:
df_2 = spark.sparkContext.parallelize([http])
df = spark.read.json(df_2)
df.show()

+--------------------+--------------------+--------------------+--------------------+-------+
|           Countries|                Date|              Global|                  ID|Message|
+--------------------+--------------------+--------------------+--------------------+-------+
|[[Afghanistan, AF...|2022-03-19T00:22:...|[2022-03-19T00:22...|8d657c6c-67c2-4c8...|       |
+--------------------+--------------------+--------------------+--------------------+-------+



In [10]:
df.printSchema()

root
 |-- Countries: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Country: string (nullable = true)
 |    |    |-- CountryCode: string (nullable = true)
 |    |    |-- Date: string (nullable = true)
 |    |    |-- ID: string (nullable = true)
 |    |    |-- NewConfirmed: long (nullable = true)
 |    |    |-- NewDeaths: long (nullable = true)
 |    |    |-- NewRecovered: long (nullable = true)
 |    |    |-- Slug: string (nullable = true)
 |    |    |-- TotalConfirmed: long (nullable = true)
 |    |    |-- TotalDeaths: long (nullable = true)
 |    |    |-- TotalRecovered: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- Global: struct (nullable = true)
 |    |-- Date: string (nullable = true)
 |    |-- NewConfirmed: long (nullable = true)
 |    |-- NewDeaths: long (nullable = true)
 |    |-- NewRecovered: long (nullable = true)
 |    |-- TotalConfirmed: long (nullable = true)
 |    |-- TotalDeaths: long (nullable = true)
 |    |--

In [11]:
df.withColumn('Countries',sf.explode(sf.col('Countries'))).select(['Countries.Country','Countries.CountryCode','Countries.Date','Countries.ID','Countries.NewConfirmed','Countries.NewDeaths','Countries.NewRecovered','Countries.TotalConfirmed','Countries.TotalDeaths','Countries.TotalRecovered']).show()

+-------------------+-----------+--------------------+--------------------+------------+---------+------------+--------------+-----------+--------------+
|            Country|CountryCode|                Date|                  ID|NewConfirmed|NewDeaths|NewRecovered|TotalConfirmed|TotalDeaths|TotalRecovered|
+-------------------+-----------+--------------------+--------------------+------------+---------+------------+--------------+-----------+--------------+
|        Afghanistan|         AF|2022-03-19T00:22:...|b4ddfb0a-22b0-4c4...|         175|        0|           0|        176918|       7651|             0|
|            Albania|         AL|2022-03-19T00:22:...|d46d4ff8-7735-426...|          81|        0|           0|        272885|       3487|             0|
|            Algeria|         DZ|2022-03-19T00:22:...|8b46d537-d7da-493...|          15|        1|           0|        265511|       6870|             0|
|            Andorra|         AD|2022-03-19T00:22:...|fddebe73-776a-441...| 